# Agent 创建方式

本 notebook 介绍 LangChain 中创建 agent 的不同方法。

## 公用代码

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
import os
from dotenv import load_dotenv

load_dotenv()
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

@tool
def search(query: str) -> str:
    """搜索信息"""
    return f"搜索'{query}'的结果：3个相关网页"

@tool
def calculator(expression: str) -> str:
    """计算表达式"""
    return str(eval(expression))

tools = [search, calculator]

## create_agent

### 通过 模型型号 创建

In [ ]:
from langchain.agents import create_agent

agent = create_agent("gpt-4o") # 支持自动推断，存在映射配置

result = agent.invoke({"messages": [HumanMessage(content="计算(10+5)*2")]})
print(result["messages"][-1].content)

计算结果是30。


### 通过 模型对象 创建

In [ ]:
from langchain.agents import create_agent

model = ChatOpenAI(model="gpt-4o")

agent = create_agent(model)

result = agent.invoke({"messages": [HumanMessage(content="计算(10+5)*2")]})
print(result["messages"][-1].content)

计算结果是：30


## 动态使用不同的模型

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

# 基础模型
basic_model = ChatOpenAI(model="gpt-4o-mini")
# 高级模型
advanced_model = ChatOpenAI(model="gpt-4o")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    messages = request.state["messages"]
    
    # 获取最后一条消息内容的长度
    if messages:

        last_message = messages[-1]
        last_message_content = last_message.content if hasattr(last_message, 'content') else str(last_message)
        print(last_message_content)
        content_length = len(last_message_content)
        
        print(f"最后一条消息长度: {content_length}")
        
        # 可以根据内容长度选择模型
        if content_length > 10:  # 长消息使用高级模型
            model = advanced_model
            print("使用高级模型 gpt-4o")
        else:
            model = basic_model
            print("使用基础模型 gpt-4o-mini")
    else:
        model = basic_model
    
    return handler(request.override(model=model))

agent = create_agent(
    model=basic_model,  # 默认模型
    middleware=[dynamic_model_selection]
)

result = agent.invoke({"messages": [HumanMessage(content="计算(10+5)*2")]})
print(result["messages"][-1].content)

result = agent.invoke({"messages": [HumanMessage(content="计算 10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10")]})
print(result["messages"][-1].content)

计算(10+5)*2
最后一条消息长度: 10
使用基础模型 gpt-4o-mini
30
最后一条消息长度: 2
使用基础模型 gpt-4o-mini
计算结果是 30。
计算 10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10+10
最后一条消息长度: 59
使用高级模型 gpt-4o
190
最后一条消息长度: 3
使用基础模型 gpt-4o-mini
计算的结果是 190。


## 2. create_tool_calling_agent

In [17]:
from langgraph.prebuilt import create_tool_calling_agent

# 创建
agent = create_tool_calling_agent(model, tools)

# 使用
result = agent.invoke({
    "messages": [HumanMessage(content="上海天气和股票价格")]
})
print("Tool Agent:", result["messages"][-1].content)

ImportError: cannot import name 'create_tool_calling_agent' from 'langgraph.prebuilt' (c:\ProgramData\miniconda3\envs\py-3.13.10\Lib\site-packages\langgraph\prebuilt\__init__.py)

## 3. 使用 create_structured_chat_agent

In [ ]:
from langchain.agents import create_structured_chat_agent

# 创建 agent（支持结构化输出）
agent = create_structured_chat_agent(model, [get_weather], prompt)
executor = AgentExecutor(agent=agent, tools=[get_weather], verbose=True)
result = executor.invoke({"input": "广州今天的天气怎么样？"})
print(result)

## 4. 使用 LangGraph 创建 Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

@tool
def calculator(expression: str) -> str:
    """简单的计算器工具，计算数学表达式"""
    try:
        result = eval(expression)
        return f"计算结果: {result}"
    except:
        return "计算错误，请检查表达式"

# 使用 LangGraph 创建 React agent
langgraph_agent = create_react_agent(model, [get_weather, calculator])

# 测试
response = langgraph_agent.invoke({"messages": [("human", "计算 123 + 456")]})
print(response["messages"][-1].content)

## 5. 使用自定义 Agent

In [ ]:
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.tools import BaseTool
from typing import List, Tuple

class CustomAgent:
    def __init__(self, model, tools, prompt):
        self.model = model
        self.tools = {tool.name: tool for tool in tools}
        self.prompt = prompt
    
    def plan(self, inputs):
        """规划执行步骤"""
        response = self.model.invoke(self.prompt.format_messages(**inputs))
        return response.content
    
    def execute(self, plan):
        """执行计划"""
        # 简化的执行逻辑
        return f"执行计划: {plan}"

# 创建自定义 agent
custom_agent = CustomAgent(model, [get_weather], prompt)
plan = custom_agent.plan({"input": "深圳今天的天气怎么样？"})
result = custom_agent.execute(plan)
print(result)

## 6. 使用 AgentType 枚举

In [ ]:
from langchain.agents import AgentType, initialize_agent

# 使用内置 AgentType
agent = initialize_agent(
    tools=[get_weather],
    llm=model,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

result = agent.run("天津今天的天气怎么样？")
print(result)